In [82]:
import os.path

import requests
import pandas as pd
from tqdm import tqdm
from typing import Any, Dict, List, Optional

parent_dir = os.path.dirname(os.getcwd())

In [46]:
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")
GROUP_NAME = "overhear"
VK_API_URL = "https://api.vk.com/method/{method}"

In [47]:
posts_df = pd.read_csv(os.path.join(parent_dir, "datasets/posts.csv"))

In [48]:
posts_df.head()

,id,text,likes,reposts,views
0,1632037,Однажды делал своей девушке куни. В самый моме...,19,34,5518
1,1632031,Живу с молодым человеком и его мамой (временна...,369,64,53145
2,1632022,"Всегда презирала проституток, порноактрис. А п...",167,47,27266
3,1632004,"Подруга полгода пролежала в коме после аварии,...",307,184,71438
4,1631999,"Когда вернулся из армии, моя бабушка умерла и ...",820,9,97900


In [49]:
posts_id = posts_df["id"].to_list()

In [92]:
group_info = requests.get(url=VK_API_URL.format(method="groups.getById"),
                          params={
                              "v": 5.131,
                              "access_token": ACCESS_TOKEN,
                              "group_ids": GROUP_NAME,
                              "fields": "description",
                          })

In [93]:
group_id = group_info.json()["response"][0]["id"]

In [94]:
info = requests.get(url=VK_API_URL.format(method="wall.getComments"),
                    params={
                        "v": 5.131,
                        "access_token": ACCESS_TOKEN,
                        "owner_id": -group_id,
                        "post_id": posts_id[0],
                        "count": 100,
                        "offset": 0,
                        "need_likes": 1,
                        "extended": 1,
                    })

In [95]:
info.json()

{'response': {'count': 13,
  'items': [{'id': 1632039,
    'from_id': 44312227,
    'date': 1649575903,
    'text': 'Бывает😸',
    'post_id': 1632037,
    'owner_id': -34215577,
    'parents_stack': [],
    'donut': {'is_don': True},
    'likes': {'can_like': 1, 'count': 1, 'user_likes': 0},
    'thread': {'count': 0,
     'items': [],
     'can_post': False,
     'show_reply_button': False}},
   {'id': 1632040,
    'from_id': 3187747,
    'date': 1649575945,
    'text': 'Какое запоминающееся событие.. Ну пукнула, значит полность доверяет вам, расслабилась.',
    'post_id': 1632037,
    'owner_id': -34215577,
    'parents_stack': [],
    'donut': {'is_don': True},
    'likes': {'can_like': 1, 'count': 118, 'user_likes': 0},
    'thread': {'count': 0,
     'items': [],
     'can_post': False,
     'show_reply_button': False}},
   {'id': 1632042,
    'from_id': 41443339,
    'date': 1649576057,
    'text': "Добро пожаловать в реальный мир.\n\nЭто вы ещё про существование 'выпуков' не в к

In [54]:
items = info.json()["response"]["items"]
item = items[0]
profile_lists = info.json()["response"]["profiles"]

In [55]:
def get_user(profiles: List[Dict[str, Any]], user_id: int) -> Dict[str, Any]:
    for profile in profiles:
        if profile["id"] == user_id:
            return profile

In [56]:
profile = get_user(profile_lists, item["from_id"])
profile

{'id': 44312227,
 'first_name': 'Елена',
 'last_name': 'Князева',
 'can_access_closed': False,
 'is_closed': True,
 'sex': 1,
 'screen_name': 'alen_knyazeva',
 'photo_50': 'https://sun7-6.userapi.com/s/v1/ig2/vrEkUkD7oUhM-UFJ0s-6rQE5SdSRT0K5JdE-oIdLWn9zkpEFdBz4LJe3vCjL69IN1T5ijQdvt72LUXYzaoFq0YVX.jpg?size=50x50&quality=95&crop=197,194,406,406&ava=1',
 'photo_100': 'https://sun7-6.userapi.com/s/v1/ig2/0I9b7ZtO4jOC1O6MWmCRD_EjQt604z6adQ-zIorMvZtbzwzksNTU9x3JWhNDJLpp1ng4YHkK4pGJeXCtiDP_SlTY.jpg?size=100x100&quality=95&crop=197,194,406,406&ava=1',
 'online_info': {'visible': True,
  'last_seen': 1649587795,
  'is_online': True,
  'app_id': 2274003,
  'is_mobile': True},
 'online': 1,
 'online_mobile': 1,
 'online_app': 2274003}

In [57]:
def get_sex(sex_value: int) -> Optional[str]:
    if sex_value == 1:
        return "w"
    if sex_value == 2:
        return "m"
    return None

In [58]:
comment_sample_dict = {
    "id": item["id"],
    "post_id": item["post_id"],
    "text": item["text"],
    "likes": item["likes"]["count"],
    "user_id": item["from_id"],
    "user_first_name": profile["first_name"],
    "user_last_name": profile["last_name"],
    "user_sex": get_sex(profile["sex"]),
}

In [59]:
comment_sample_dict

{'id': 1632039,
 'post_id': 1632037,
 'text': 'Бывает😸',
 'likes': 1,
 'user_id': 44312227,
 'user_first_name': 'Елена',
 'user_last_name': 'Князева',
 'user_sex': 'w'}

In [60]:
comments = []

In [84]:
%%time
for post_id in tqdm(posts_id):
    response = requests.get(url=VK_API_URL.format(method="wall.getComments"),
                    params={
                        "v": 5.131,
                        "access_token": ACCESS_TOKEN,
                        "owner_id": -group_id,
                        "post_id": post_id,
                        "count": 100,
                        "offset": 0,
                        "need_likes": 1,
                        "extended": 1,
                    })
    if response.status_code != 200:
        break
    try:
        items = response.json()["response"]["items"]
        profiles = info.json()["response"]["profiles"]
        for item in items:
            profile = get_user(profiles, item["from_id"])
            comment_dict = {
                "id": item["id"],
                "post_id": item["post_id"],
                "text": item["text"],
                "likes": item["likes"]["count"],
                "user_id": item["from_id"],
                "user_first_name": profile["first_name"] if profile else None,
                "user_last_name": profile["last_name"] if profile else None,
                "user_sex": get_sex(profile["sex"]) if profile else None,
                "is_mobile": profile["online_info"]["is_mobile"] if profile else None
            }
            comments.append(comment_dict)
    except KeyError:
        continue

 54%|█████▍    | 47521/87994 [1:58:53<1:41:15,  6.66it/s] 

CPU times: total: 11min 56s
Wall time: 1h 58min 53s


In [85]:
comments[0:10]

[{'id': 1632039,
  'post_id': 1632037,
  'text': 'Бывает😸',
  'likes': 1,
  'user_id': 44312227,
  'user_first_name': 'Елена',
  'user_last_name': 'Князева',
  'user_sex': 'w'},
 {'id': 1632040,
  'post_id': 1632037,
  'text': 'Какое запоминающееся событие.. Ну пукнула, значит полность доверяет вам, расслабилась.',
  'likes': 102,
  'user_id': 3187747,
  'user_first_name': 'Сергей',
  'user_last_name': 'Приходько',
  'user_sex': 'm'},
 {'id': 1632042,
  'post_id': 1632037,
  'text': "Добро пожаловать в реальный мир.\n\nЭто вы ещё про существование 'выпуков' не в курсе.",
  'likes': 171,
  'user_id': 41443339,
  'user_first_name': 'Анна',
  'user_last_name': 'Пашкова',
  'user_sex': 'w'},
 {'id': 1632035,
  'post_id': 1632031,
  'text': 'Ага, вертолёты, мама ж глупая😏',
  'likes': 46,
  'user_id': 290338758,
  'user_first_name': None,
  'user_last_name': None,
  'user_sex': None},
 {'id': 1632036,
  'post_id': 1632031,
  'text': 'Это когда пукнул в детстве, а потом делаешь так губами, ч

In [87]:
comment_df = pd.DataFrame(data=comments)

In [89]:
comment_df.head()

,id,post_id,text,likes,user_id,user_first_name,user_last_name,user_sex,is_mobile
0,1632039,1632037,Бывает😸,1,44312227,Елена,Князева,w,NaN
1,1632040,1632037,"Какое запоминающееся событие.. Ну пукнула, зна...",102,3187747,Сергей,Приходько,m,NaN
2,1632042,1632037,Добро пожаловать в реальный мир.\n\nЭто вы ещё...,171,41443339,Анна,Пашкова,w,NaN
3,1632035,1632031,"Ага, вертолёты, мама ж глупая😏",46,290338758,None,None,None,NaN
4,1632036,1632031,"Это когда пукнул в детстве, а потом делаешь та...",245,525157090,None,None,None,NaN


In [91]:
comment_df.to_csv(os.path.join(parent_dir, "datasets/comments.csv"), index=False)